In [2]:
%pip install scikit-learn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------- -------------------------- 20.5/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
    --------------------------------------- 0.2/8.9 MB 4.5 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/8.9 MB 6.6 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/8.9 MB 10.4 MB/s eta 0:00:01
   ----- ---------------------------------- 1.2/8.9 MB 7.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/8.9 MB 7.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/8.9 MB 7.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/8.9 MB 7.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/8.9 MB 7.4 MB/s eta 0:00:02
   ----- -------------------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 640.0 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 538.3 kB/s eta 0:00:00
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB 8.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.8/12.9 MB 22.0 MB/s eta 0:00:01
   -------- ------------------------------- 2.7/12.9 MB 21.4 MB/s eta 0:00:01
   ---------- ----------------------------- 3.3/12.9 MB 19.1 MB/s eta 0:00:01
   ---------- ----------------------------- 3.5/12.9 MB 15.9 MB/s eta 0:00:01
   ----------- ---------------------------- 3.7/12.9 MB 14.1 MB/s eta 0:00:01


  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:

import pickle as pkl
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors
import cv2

In [6]:
feature_list = np.array(pkl.load(open('feature_list.pkl','rb')))
filenames = pkl.load(open('filenames.pkl','rb'))

In [7]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

# Create a Sequential model and add GlobalMaxPooling2D layer
seq_model = Sequential([model, GlobalMaxPooling2D()])
print(seq_model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 2048)           │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

None


In [8]:
img = image.load_img('images/1163.jpg', target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [9]:
neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
neighbors.fit(feature_list)

,n_neighbors,6
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'euclidean'
,p,2
,metric_params,None
,n_jobs,None


In [11]:
# Extract features using seq_model to get 2048 features
result = seq_model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

distances, indices = neighbors.kneighbors([normalized_result])
print(indices)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[[ 22 504 645   5 557 658]]


In [12]:
for file in indices[0]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(temp_img, (512, 512)))
    cv2.waitKey(0)

cv2.destroyAllWindows()